# Used Car Practice

I wrote a paper in an internal publication called Quality Quarterly some years ago (sometime between 2015-2018) that examined the homogeneity of Used Car prices in the published Consumer Price Index areas relative to a study of microdata over a period of about six months.  I called the a "single car index."  I believe it was a 2013 Chevy Malibu.  I collected the data by visiting the Carmax website each month and following the same car or the nearest-neighbor vehicle and collecting that into excel.

In the past few years, I've wanted to redo that study but using better methods.  Something involving webscraping or an API pull would be the goal.

I don't know of many sources, at least that are free, that have unsummarized raw U.S. auto sales.  Taking them from a website like Carmax allows you to create a list price based index.

While looking for sources as a long-shot, I stumbled into this Craiglist used car dataset that is hosted by Kaggle.

In [1]:
import pandas as pd
import numpy as np

### Importing Craigslist Data
##### source = https://www.kaggle.com/datasets/austinreese/craigslist-carstrucks-data?resource=download

In [2]:
craigslist_raw = pd.read_csv('vehicles.csv')

craigslist_raw.head(2)

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
0,7222695916,https://prescott.craigslist.org/cto/d/prescott...,prescott,https://prescott.craigslist.org,6000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN,NaN
1,7218891961,https://fayar.craigslist.org/ctd/d/bentonville...,fayetteville,https://fayar.craigslist.org,11900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN,NaN


### Examining Craiglist Data Briefly

In [3]:
# Looking at rows to find which ones have the most info in them.
craigslist_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 26 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   url           426880 non-null  object 
 2   region        426880 non-null  object 
 3   region_url    426880 non-null  object 
 4   price         426880 non-null  int64  
 5   year          425675 non-null  float64
 6   manufacturer  409234 non-null  object 
 7   model         421603 non-null  object 
 8   condition     252776 non-null  object 
 9   cylinders     249202 non-null  object 
 10  fuel          423867 non-null  object 
 11  odometer      422480 non-null  float64
 12  title_status  418638 non-null  object 
 13  transmission  424324 non-null  object 
 14  VIN           265838 non-null  object 
 15  drive         296313 non-null  object 
 16  size          120519 non-null  object 
 17  type          334022 non-null  object 
 18  pain

In [4]:
# I thought it was interesting that the number of non-nulls was bigger for model than manufacturer.
#   Honda the manufacturer is the parent in the hierarchy above Civic the model.

# This is looking at the ones where the model is known but the manufacturer is not
craigslist_raw.loc[(~craigslist_raw['model'].isnull() & craigslist_raw['manufacturer'].isnull()), ['manufacturer', 'model']]

,manufacturer,model
97,NaN,Scion iM Hatchback 4D
122,NaN,blue bird bus
135,NaN,Scion iM Hatchback 4D
137,NaN,1966 C-30 1 ton
155,NaN,smart fortwo Passion Hatchback
...,...,...
426843,NaN,2017
426846,NaN,2015
426862,NaN,Genesis G80 3.8 Sedan 4D
426863,NaN,Genesis G80 3.8 Sedan 4D


In [5]:
# This is looking at the ones where the manufacturer is known but the model is not
craigslist_raw.loc[(craigslist_raw['model'].isnull() & ~craigslist_raw['manufacturer'].isnull()), ['manufacturer', 'model']]

,manufacturer,model
96,honda,NaN
101,volvo,NaN
280,chevrolet,NaN
308,rover,NaN
414,lincoln,NaN
...,...,...
426603,ram,NaN
426606,ram,NaN
426614,ram,NaN
426618,ram,NaN


### Getting Craigslist Areas that Correspond to Published Used Car CPI Areas

#### Top Craigslist Used Car Markets

In [6]:
key_col = ['id']
groupby_col = ['region']

region_count = craigslist_raw[key_col + groupby_col].groupby(groupby_col).count().rename(columns={"id":"count"})
region_count.sort_values(by='count', ascending=False).head(10)

,count
region,
columbus,3608
jacksonville,3562
spokane / coeur d'alene,2988
eugene,2985
fresno / madera,2983
orlando,2983
omaha / council bluffs,2982
bend,2982
new hampshire,2981


We can see that Craigslist entries are not necessarily correlated with market size.  (No New York, LA, Chicago, DFW, Houston, etc in the top 10)

#### Published areas for CPI
- New York-Newark-Jersey City, NY-NJ-PA
- Philadelphia-Camden-Wilmington, PA-NJ-DE-MD
- Boston-Cambridge-Newton, MA-NH
- Chicago-Naperville-Elgin, IL-IN-WI
- Detroit-Warren-Dearborn, MI
- Minneapolis-St. Paul-Bloomington, MN-WI
- St. Louis, MO-IL
- Atlanta-Sandy Springs-Roswell, GA
- Dallas-Fort Worth-Arlington, TX
- Houston-The Woodlands-Sugar Land, TX
- Miami-Fort Lauderdale-West Palm Beach, FL
- Tampa-St. Petersburg-Clearwater, FL
- Washington-Arlington-Alexandria, DC-VA-MD-WV
- Baltimore-Columbia-Towson, MD
- Urban Alaska
- Denver-Aurora-Lakewood, CO
- Urban Hawaii
- Los Angeles-Long Beach-Anaheim, CA
- Riverside-San Bernardino-Ontario, CA
- Phoenix-Mesa-Scottsdale, AZ
- San Diego-Carlsbad, CA
- San Francisco-Oakland-Hayward, CA
- Seattle-Tacoma-Bellevue, WA

https://www.bls.gov/cpi/questions-and-answers.htm#Question_17

In [7]:
# These are areas that are published for used cars.
cpi_areas = ['New York-Newark-Jersey City, NY-NJ-PA', 'Philadelphia-Camden-Wilmington, PA-NJ-DE-MD', 'Boston-Cambridge-Newton, MA-NH', 'Chicago-Naperville-Elgin, IL-IN-WI', 'Detroit-Warren-Dearborn, MI', 'Minneapolis-St. Paul-Bloomington, MN-WI', 'St. Louis, MO-IL', 'Atlanta-Sandy Springs-Roswell, GA', 'Dallas-Fort Worth-Arlington, TX', 'Houston-The Woodlands-Sugar Land, TX', 'Miami-Fort Lauderdale-West Palm Beach, FL', 'Tampa-St. Petersburg-Clearwater, FL', 'Washington-Arlington-Alexandria, DC-VA-MD-WV', 'Baltimore-Columbia-Towson, MD', 'Urban Alaska', 'Denver-Aurora-Lakewood, CO', 'Urban Hawaii', 'Los Angeles-Long Beach-Anaheim, CA', 'Riverside-San Bernardino-Ontario, CA', 'Phoenix-Mesa-Scottsdale, AZ', 'San Diego-Carlsbad, CA', 'San Francisco-Oakland-Hayward, CA', 'Seattle-Tacoma-Bellevue, WA']
cpi_areas_df = pd.DataFrame(cpi_areas).rename(columns={0:"longform_psu_name"})
cpi_areas_df.head(2)

,longform_psu_name
0,"New York-Newark-Jersey City, NY-NJ-PA"
1,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD"


In [8]:
# # Looking at New York
# region_count.loc[region_count.index.str.contains('new york'), :]

In [9]:
# # Looking at Philadelphia
# region_count.loc[region_count.index.str.contains('phil'), :]

In [10]:
# # This was how I tried to find the areas, but I'm going to comment it out to focus.

# for area in cpi_areas_df.iloc[:,0]:
#     print('\n')
#     num_char_to_search = 5
#     print("For PSU {0}, the first {1} characters from that are:".format(area, num_char_to_search))
#     area_root = area.lower()[0:num_char_to_search]
#     print('\n')
#     print(area_root)
#     print(region_count.loc[region_count.index.str.contains(area_root), :])

This quick loop gave the answer to us for all but these six.
- St. Louis, MO-IL
- Miami-Fort Lauderdale-West Palm Beach, FL
- Urban Alaska
- Urban Hawaii
- Riverside-San Bernardino-Ontario, CA
- San Francisco-Oakland-Hayward, CA

In [11]:
# # I guessed some and poked around Craigslist for the others.
# try_these = ["st lo", "south flo", "ancho", "hawai", "inlan", "SF "]
# # This worked, but I'm going to comment it out to focus.
# for area in try_these:
#     area_root = area
#     print('\n')
#     print(area_root)
#     print(region_count.loc[region_count.index.str.contains(area_root), :])

In [12]:
# I looked and found the row numbers for the problem cases and separated them.
easy_areas = cpi_areas_df.drop(index=[6, 10, 14, 16, 18, 21])
hard_areas = cpi_areas_df.iloc[[6, 10, 14, 16, 18, 21], :]

area_dict = {}
for area in easy_areas.iloc[:,0]:
    num_char_to_search = 5
    area_root = area.lower()[0:num_char_to_search]
    possible_match = region_count.loc[region_count.index.str.contains(area_root), :]
    area_dict[area]=possible_match.index[0]

try_these = ["st lo", "south flo", "ancho", "hawai", "inlan", "SF "]
counter = 0
for area in hard_areas.iloc[:,0]:
    area_root = try_these[counter]
    possible_match = region_count.loc[region_count.index.str.contains(area_root), :]
    area_dict[area]=possible_match.index[0]
    counter = counter + 1
    
area_dict

{'New York-Newark-Jersey City, NY-NJ-PA': 'new york city',
 'Philadelphia-Camden-Wilmington, PA-NJ-DE-MD': 'philadelphia',
 'Boston-Cambridge-Newton, MA-NH': 'boston',
 'Chicago-Naperville-Elgin, IL-IN-WI': 'chicago',
 'Detroit-Warren-Dearborn, MI': 'detroit metro',
 'Minneapolis-St. Paul-Bloomington, MN-WI': 'minneapolis / st paul',
 'Atlanta-Sandy Springs-Roswell, GA': 'atlanta',
 'Dallas-Fort Worth-Arlington, TX': 'dallas / fort worth',
 'Houston-The Woodlands-Sugar Land, TX': 'houston',
 'Tampa-St. Petersburg-Clearwater, FL': 'tampa bay area',
 'Washington-Arlington-Alexandria, DC-VA-MD-WV': 'washington, DC',
 'Baltimore-Columbia-Towson, MD': 'baltimore',
 'Denver-Aurora-Lakewood, CO': 'denver',
 'Los Angeles-Long Beach-Anaheim, CA': 'los angeles',
 'Phoenix-Mesa-Scottsdale, AZ': 'phoenix',
 'San Diego-Carlsbad, CA': 'san diego',
 'Seattle-Tacoma-Bellevue, WA': 'seattle-tacoma',
 'St. Louis, MO-IL': 'st louis',
 'Miami-Fort Lauderdale-West Palm Beach, FL': 'south florida',
 'Urban 

#### Subset craigslist dataframe to just the CPI published used cars areas.

In [13]:
craigslist_raw.head(1)

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
0,7222695916,https://prescott.craigslist.org/cto/d/prescott...,prescott,https://prescott.craigslist.org,6000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN,NaN


In [29]:
craigslist_reduced = craigslist_raw.loc[craigslist_raw['region'].isin(area_dict.values()),:]
n1 = craigslist_raw.shape[0]
n2 = craigslist_reduced.shape[0]
msg = """The craigslist dataset from the universe of all areas had {0} rows.
The reduced craigslist dataset from just the CPI published used cars areas has {1} rows.""".format(n1,n2)
print(msg)

The craigslist dataset from the universe of all areas had 426880 rows.
The reduced craigslist dataset from just the CPI published used cars areas has 64867 rows.


In [30]:
# I'll take a quick look at the reduced data set.
craigslist_reduced.info()
print("It looks like we should drop county from the dataframe since it is always null here.")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64867 entries, 4982 to 409953
Data columns (total 26 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            64867 non-null  int64  
 1   url           64867 non-null  object 
 2   region        64867 non-null  object 
 3   region_url    64867 non-null  object 
 4   price         64867 non-null  int64  
 5   year          64628 non-null  float64
 6   manufacturer  61938 non-null  object 
 7   model         64125 non-null  object 
 8   condition     36997 non-null  object 
 9   cylinders     38584 non-null  object 
 10  fuel          64512 non-null  object 
 11  odometer      64111 non-null  float64
 12  title_status  63618 non-null  object 
 13  transmission  64391 non-null  object 
 14  VIN           37776 non-null  object 
 15  drive         44747 non-null  object 
 16  size          20326 non-null  object 
 17  type          51318 non-null  object 
 18  paint_color   46177 no

In [31]:
c1 = craigslist_reduced.shape[1]
craigslist_reduced = craigslist_reduced.drop(columns=['county'])
c2 = craigslist_reduced.shape[1]
print("Number of columns was {0}; now is {1}.".format(c1, c2))

Number of columns was 26; now is 25.


###  Quick version steps
#### I'm going to drop the rows where we are missing manufacturer or model or both.  It's possible that some of them could be inferred from non-missing information, but for this first version, I'm simply going to drop them.

#### Aside of some possibly salvageable rows

In [53]:
# This is looking at the ones where the model is known but the manufacturer is not
#  Some models have easy-to-identify manufacturers.  F350 = Ford.  Wrangler = Jeep.  Geo Prizm = Geo.
df = craigslist_reduced
n = df.loc[(~df['model'].isnull() & df['manufacturer'].isnull()), ['manufacturer', 'model']].shape[0]
print("There are {} rows like this.\n".format(n))
df.loc[(~df['model'].isnull() & df['manufacturer'].isnull()), ['manufacturer', 'model']].head(4)

There are 2916 rows like this.



,manufacturer,model
4996,NaN,F350 DRW
5003,NaN,freightliner m2 106
5007,NaN,2019 wrangler
5009,NaN,Geo Prizm


In [54]:
# This is looking at the ones where the manufacturer is known but the model is not.
#  Some single-model manufacturers might allow for filling out the model.
df = craigslist_reduced
n = df.loc[(df['model'].isnull() & ~df['manufacturer'].isnull()), ['manufacturer', 'model']].shape[0]
print("There are {} rows like this.\n".format(n))
df.loc[(df['model'].isnull() & ~df['manufacturer'].isnull()), ['manufacturer', 'model']].head(4)

There are 729 rows like this.



,manufacturer,model
4990,ram,NaN
5123,saturn,NaN
5178,chrysler,NaN
5272,ford,NaN


In [55]:
n1 = craigslist_reduced.shape[0]
craigslist_reduced = craigslist_reduced.dropna(subset=['manufacturer', 'model'])
n2 = craigslist_reduced.shape[0]
print("Number of rows was {0}; now is {1}.".format(n1, n2))

Number of rows was 64867; now is 61209.


In [61]:
craigslist_reduced.head(3)

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,drive,size,type,paint_color,image_url,description,state,lat,long,posting_date
4982,7316889831,https://anchorage.craigslist.org/cto/d/anchora...,anchorage / mat-su,https://anchorage.craigslist.org,55000,2013.0,ford,f450 super duty,excellent,8 cylinders,...,4wd,NaN,truck,white,https://images.craigslist.org/00L0L_2Ps0KDoijC...,"FOR SALE - 2013 Ford F450 4WD Dually, Lariat, ...",ak,61.190000,-149.89380,2021-05-04T11:31:21-0800
4983,7316889728,https://anchorage.craigslist.org/ctd/d/anchora...,anchorage / mat-su,https://anchorage.craigslist.org,46194,2020.0,ram,1500,NaN,8 cylinders,...,4wd,NaN,truck,NaN,https://images.craigslist.org/00Z0Z_fp44vWPvV9...,"2020 Ram 1500 Big Horn -- $46,194 ☎ Call...",ak,61.176984,-149.86941,2021-05-04T11:31:14-0800
4984,7316889528,https://anchorage.craigslist.org/ctd/d/anchora...,anchorage / mat-su,https://anchorage.craigslist.org,26194,2019.0,hyundai,kona,NaN,4 cylinders,...,4wd,NaN,SUV,white,https://images.craigslist.org/00L0L_dik21ehsWZ...,"2019 Hyundai KONA Limited -- $26,194 ☎ C...",ak,61.176984,-149.86941,2021-05-04T11:30:54-0800


In [72]:
# Looking at listed condition.
craigslist_reduced[['condition','id']].groupby('condition', dropna=False).count()

,id
condition,
excellent,14445
fair,945
good,15328
like new,4052
new,275
salvage,81
NaN,26083


In [81]:
beg = craigslist_reduced.posting_date.min()
end = craigslist_reduced.posting_date.max()
msg = "This dataset begins on {0}.\nThis dataset ends with {1}.\n This represents one month of data.".format(beg, end)
print(msg)

This dataset begins on 2021-04-04T07:05:53-0800.
This dataset ends with 2021-05-04T22:20:18-0400.
 This represents one month of data.


I should have looked into the data before going this far.  I thought I had years or at least a years worth of data.  Instead, I have one month.

In [83]:
# I got these values from here
# https://www.motor1.com/news/559019/most-popular-used-cars-2021/
rank = [1,2,3,4,5,6,7,8,9,10]
vehicle = ['Ford F-150', 'Chevrolet Silverado 1500', 'Ram 1500', 'Honda Civic',
           'Nissan Rogue', 'Toyota Camry', 'Chevrolet Equinox', 'Ford Escape', 
           'Toyota RAV4', 'Honda Accord']
share_of_used_car_sales = [.036, .026, .024, .020, .019, .019, .019, .017, .017, .016]

In [93]:
best_selling_2021 = pd.DataFrame({'rank': rank, 'vehicle': vehicle, 'share_of_used_car_sales': share_of_used_car_sales})
best_selling_2021.style.format().hide(axis="index")

rank,vehicle,share_of_used_car_sales
1,Ford F-150,0.036000
2,Chevrolet Silverado 1500,0.026000
3,Ram 1500,0.024000
4,Honda Civic,0.020000
5,Nissan Rogue,0.019000
6,Toyota Camry,0.019000
7,Chevrolet Equinox,0.019000
8,Ford Escape,0.017000
9,Toyota RAV4,0.017000
10,Honda Accord,0.016000


In [101]:
df = craigslist_reduced

#df.loc[df['model'].str.contains('f150'), :].shape[0]
#df.loc[df['model'].str.contains('f-150'), :].shape[0]
df.loc[df['model'].str.contains('f-150') | df['model'].str.contains('f150'), :].shape[0]

1889

In [102]:
df = craigslist_reduced

df.loc[df['model'].str.contains('silverado 1500'), :].shape[0]

950

In [103]:
df = craigslist_reduced

df.loc[df['model'].str.contains('Silverado 1500'), :].shape[0]

0

In [104]:
df = craigslist_reduced

df.loc[df['model'].str.contains('silver'), :]

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,drive,size,type,paint_color,image_url,description,state,lat,long,posting_date
4988,7316865395,https://anchorage.craigslist.org/ctd/d/anchora...,anchorage / mat-su,https://anchorage.craigslist.org,31995,2013.0,chevrolet,silverado,NaN,NaN,...,4wd,NaN,pickup,white,https://images.craigslist.org/00P0P_cIKtSJcHn7...,2013 Chevrolet Chevy Silverado 2500HD Work Tru...,ak,61.161670,-149.863120,2021-05-04T10:50:37-0800
4989,7316859914,https://anchorage.craigslist.org/ctd/d/anchora...,anchorage / mat-su,https://anchorage.craigslist.org,31995,2013.0,chevrolet,silverado,NaN,NaN,...,4wd,NaN,pickup,white,https://images.craigslist.org/00P0P_cIKtSJcHn7...,2013 Chevrolet Chevy Silverado 2500HD Work Tru...,ak,61.161670,-149.863120,2021-05-04T10:41:27-0800
5008,7316609695,https://anchorage.craigslist.org/ctd/d/palmer-...,anchorage / mat-su,https://anchorage.craigslist.org,13950,2007.0,chevrolet,silverado 1500,excellent,8 cylinders,...,4wd,full-size,truck,black,https://images.craigslist.org/00g0g_9PQkbUl5C1...,***CLEAN CARFAX***RUST FREE***WA STATE 4X4***W...,ak,61.480760,-148.798828,2021-05-03T22:31:01-0800
5011,7316604826,https://anchorage.craigslist.org/cto/d/palmer-...,anchorage / mat-su,https://anchorage.craigslist.org,58500,2017.0,chevrolet,silverado 2500hd,excellent,NaN,...,NaN,full-size,truck,white,https://images.craigslist.org/00y0y_fhWL6SfQIn...,Very clean Chevrolet Silverado 2500 Duramax LT...,ak,61.606968,-149.109364,2021-05-03T21:41:43-0800
5027,7316565560,https://anchorage.craigslist.org/cto/d/kenai-2...,anchorage / mat-su,https://anchorage.craigslist.org,29500,2017.0,chevrolet,silverado,NaN,NaN,...,NaN,NaN,NaN,NaN,https://images.craigslist.org/00B0B_gj5xEK33mf...,"2017 Chevrolet Silverado 1/2 ton, 4x4, V-8 aut...",ak,60.819750,-150.097890,2021-05-03T18:22:06-0800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409502,7316356402,https://seattle.craigslist.org/see/cto/d/seatt...,seattle-tacoma,https://seattle.craigslist.org,10700,1999.0,chevrolet,silverado 1500,good,8 cylinders,...,4wd,NaN,truck,red,https://images.craigslist.org/01111_4c2At41ela...,Late 99 Silverado 1500 4 wheel drive. Regular ...,wa,47.746134,-122.343186,2021-05-03T12:02:02-0700
409556,7316333192,https://seattle.craigslist.org/tac/ctd/d/lakew...,seattle-tacoma,https://seattle.craigslist.org,0,2007.0,chevrolet,silverado 1500 ltz,NaN,8 cylinders,...,4wd,NaN,NaN,NaN,https://images.craigslist.org/00E0E_irC3UzAsCn...,STOCK #: 16671A CALL: (206) 208...,wa,47.166808,-122.483522,2021-05-03T11:25:38-0700
409675,7316289736,https://seattle.craigslist.org/tac/ctd/d/rento...,seattle-tacoma,https://seattle.craigslist.org,579,2014.0,chevrolet,silverado 1500,NaN,8 cylinders,...,4wd,NaN,truck,NaN,https://images.craigslist.org/00w0w_grtDoeK3Gw...,Brotherton Cadillac Buick GMC STOCK #: ...,wa,47.469444,-122.220889,2021-05-03T10:19:05-0700
409702,7316282110,https://seattle.craigslist.org/tac/ctd/d/rento...,seattle-tacoma,https://seattle.craigslist.org,515,2014.0,chevrolet,silverado 1500,NaN,8 cylinders,...,4wd,NaN,truck,NaN,https://images.craigslist.org/00808_5rORwfnYC3...,Brotherton Cadillac Buick GMC STOCK #: ...,wa,47.469444,-122.220889,2021-05-03T10:07:29-0700


Want to take a look at the most common manufacturer and model tomorrow.

Probably want to replicate a year's worth of data using a single model and a single model year for the first pass of this project.

Then, I want to compare each area's index performance with that of it's CPI counterpart to look at the volatility of one compared with the staidness of the other.

I need to reduce the areas further to just the three monthly areas (since we only have one month of data).
- NYC
- LA
- Chicago